<a href="https://colab.research.google.com/github/kopmean/DS4Biz/blob/main/Boonmee_DA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Data

In [1]:
!gdown 1jVWjl4b2fndW3Uj0smTUnfbdhLnF9g9t
!gdown 1tfZZLnYFdtOTrlr_xD86FFGwqZzsREQ6

Downloading...
From: https://drive.google.com/uc?id=1jVWjl4b2fndW3Uj0smTUnfbdhLnF9g9t
To: /content/transaction.csv
100% 1.39M/1.39M [00:00<00:00, 37.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tfZZLnYFdtOTrlr_xD86FFGwqZzsREQ6
To: /content/user.csv
100% 15.2k/15.2k [00:00<00:00, 28.3MB/s]


In [2]:
import pandas as pd

# Load user data
user_df = pd.read_csv('user.csv')

# Load transaction data
transaction_df = pd.read_csv('transaction.csv')

In [3]:
# Merge user and transaction data
merged_df = pd.merge(transaction_df, user_df, on='user_id')
merged_df

,date,hour,user_id,province,hometown
0,2021-09-16,18:00,1,Sukhothai,Phrae
1,2021-09-17,06:00,1,Phayao,Phrae
2,2021-09-16,23:00,1,Phrae,Phrae
3,2021-09-16,11:00,1,Phrae,Phrae
4,2021-11-20,04:00,1,Phayao,Phrae
...,...,...,...,...,...
41690,2021-12-30,18:00,1000,Samut Sakhon,Bangkok
41691,2021-12-30,21:00,1000,Ratchaburi,Bangkok
41692,2021-12-31,09:00,1000,Bangkok,Bangkok
41693,2021-12-31,14:00,1000,Samut Prakan,Bangkok


In [4]:
# Sort by user_id and date
merged_df = merged_df.sort_values(by=['user_id', 'date', 'hour'])
merged_df

,date,hour,user_id,province,hometown
3,2021-09-16,11:00,1,Phrae,Phrae
0,2021-09-16,18:00,1,Sukhothai,Phrae
2,2021-09-16,23:00,1,Phrae,Phrae
1,2021-09-17,06:00,1,Phayao,Phrae
8,2021-11-19,13:00,1,Phrae,Phrae
...,...,...,...,...,...
41691,2021-12-30,21:00,1000,Ratchaburi,Bangkok
41689,2021-12-30,23:00,1000,Phetchaburi,Bangkok
41692,2021-12-31,09:00,1000,Bangkok,Bangkok
41693,2021-12-31,14:00,1000,Samut Prakan,Bangkok


# Prepare Data

In [5]:
# Identify trips
trips = []
trip_id = 1
current_trip = None

In [6]:
for _, row in merged_df.iterrows():
    if current_trip is None:
        if row['province'] != row['hometown']: #check Province != Hometown to create trip
            current_trip = {
                'trip_id': trip_id,
                'user_id': row['user_id'],
                'start_date': row['date'],
                'end_date': row['date'],
                'province_list': [row['province']]
            }
    else:
        if row['province'] == row['hometown']: #check Province = Hometown to append last Trip and change to next Trip
            trips.append(current_trip)
            current_trip = None
            trip_id += 1
        else: #check Province != Hometown
            current_trip['end_date'] = row['date']
            if row['province'] not in current_trip['province_list']:
                current_trip['province_list'].append(row['province'])

# Convert trips to DataFrame
trips_df = pd.DataFrame(trips)
trips_df['province_list'] = trips_df['province_list'].apply(lambda x: ', '.join(x))

In [7]:
trips_df

,trip_id,user_id,start_date,end_date,province_list
0,1,1,2021-09-16,2021-09-16,Sukhothai
1,2,1,2021-09-17,2021-09-17,Phayao
2,3,1,2021-11-19,2021-11-20,"Sukhothai, Lampang, Phayao, Chiang Rai"
3,4,1,2021-12-20,2021-12-20,Lampang
4,5,1,2021-12-25,2021-12-25,"Bangkok, Nonthaburi"
...,...,...,...,...,...
10825,10826,1000,2021-12-23,2021-12-23,Samut Prakan
10826,10827,1000,2021-12-24,2021-12-24,Pathum Thani
10827,10828,1000,2021-12-29,2021-12-29,Pathum Thani
10828,10829,1000,2021-12-30,2021-12-30,"Samut Sakhon, Ratchaburi, Phetchaburi"


# Answer the questions

## What is the total number of trips?

In [8]:
total_trips = len(trips_df)
print(f"Total number of trips: {total_trips}")

Total number of trips: 10830


##How many provinces are there in the trip with the most number of provinces?

In [43]:
max_provinces = trips_df['province_list'].apply(lambda x: len(x.split(', '))).max()
print(f"Maximum number of provinces in a trip: {max_provinces}")

Maximum number of provinces in a trip: 12


In [50]:
max_trips = trips_df[trips_df['num_provinces'] == max_provinces]
print(max_trips[['trip_id','num_provinces']])

      trip_id  num_provinces
6107     6108             12
7698     7699             12
8573     8574             12


In [51]:
for _, row in max_trips.iterrows():
    print(f"Trip ID: {row['trip_id']}, Number of Provinces: {row['num_provinces']}")
    print(f"Province List: {row['province_list']}")
    print()

Trip ID: 6108, Number of Provinces: 12
Province List: Chachoengsao, Phra Nakhon Si Ayutthaya, Pathum Thani, Chon Buri, Bangkok, Phetchaburi, Nonthaburi, Ratchaburi, Kanchanaburi, Ang Thong, Samut Sakhon, Suphan Buri

Trip ID: 7699, Number of Provinces: 12
Province List: Bangkok, Pathum Thani, Saraburi, Lop Buri, Nakhon Sawan, Phichit, Chai Nat, Suphan Buri, Kanchanaburi, Nakhon Pathom, Samut Sakhon, Samut Songkhram

Trip ID: 8574, Number of Provinces: 12
Province List: Lamphun, Chon Buri, Lampang, Sukhothai, Rayong, Phitsanulok, Pathum Thani, Bangkok, Samut Prakan, Samut Sakhon, Nakhon Pathom, Kanchanaburi



## What are the most common province pairs that people travel to in the same trip?

In [62]:
from itertools import combinations
from collections import Counter

province_pairs = []
most_n = 10

for trip in trips_df['province_list']:
    provinces = trip.split(', ')
    pairs = list(combinations(provinces, 2))
    # print(pairs)
    province_pairs.extend(pairs)

most_common_pairs = Counter(province_pairs).most_common(most_n)
# print(f"Most common province pair: {most_common_pairs[0][0]} with {most_common_pairs[0][1]} occurrences")

# Display top 10 pairs
print(f"Top {most_n} most common province pairs:")
for i, (pair, count) in enumerate(most_common_pairs, start=1):
    print(f"{i}. {pair[0]} -> {pair[0]}: {count} occurrences")

Top 10 most common province pairs:
1. Pathum Thani -> Pathum Thani: 351 occurrences
2. Pathum Thani -> Pathum Thani: 302 occurrences
3. Pathum Thani -> Pathum Thani: 291 occurrences
4. Samut Prakan -> Samut Prakan: 277 occurrences
5. Bangkok -> Bangkok: 268 occurrences
6. Bangkok -> Bangkok: 267 occurrences
7. Bangkok -> Bangkok: 265 occurrences
8. Samut Prakan -> Samut Prakan: 239 occurrences
9. Nonthaburi -> Nonthaburi: 238 occurrences
10. Nakhon Pathom -> Nakhon Pathom: 236 occurrences


## How can we use the data to support our client’s business?

### 1. การให้ Promotion แก่ลูกค้าที่เดินทางบ่อย (Frequent Travelers)

*   **สร้างโปรแกรมสะสมแต้มหรือสิทธิพิเศษ**:
สำหรับลูกค้าที่เดินทางบ่อย เช่น การสะสมไมล์หรือแต้มเพื่อแลกของรางวัลหรือส่วนลดในการเดินทางครั้งต่อไป

*   **ส่วนลดพิเศษ**: ให้ส่วนลดหรือสิทธิพิเศษสำหรับลูกค้าที่เดินทางบ่อย เพื่อสร้างความภักดี (loyalty) และกระตุ้นให้กลับมาใช้บริการอีก





In [67]:
# Example: Identifying Frequent travelers
# การระบุลูกค้าที่เดินทางบ่อยครั้งสามารถช่วยในการสร้างโปรแกรม loyalty หรือเสนอส่วนลดพิเศษเพื่อรักษาลูกค้าที่มีคุณค่าเหล่านี้
frequent_travelers = trips_df['user_id'].value_counts().head(5)
print("Top 5 frequent travelers:")
print(frequent_travelers)
print()
print("โปรโมชั่นสำหรับลูกค้าที่เดินทางบ่อย:")
print("User 857 ได้รับส่วนลดพิเศษสำหรับการเดินทางบ่อยไปยัง Chiang Mai")
print("User 732 ได้รับส่วนลดพิเศษสำหรับการเดินทางบ่อยไปยัง Bangkok")

Top 5 frequent travelers:
user_id
857    29
732    25
771    25
902    25
96     24
Name: count, dtype: int64

โปรโมชั่นสำหรับลูกค้าที่เดินทางบ่อย:
User 857 ได้รับส่วนลดพิเศษสำหรับการเดินทางบ่อยไปยัง Chiang Mai
User 732 ได้รับส่วนลดพิเศษสำหรับการเดินทางบ่อยไปยัง Bangkok


### 2. การโฆษณาร่วมกับร้านค้าในจังหวัดยอดนิยม


*   **สร้างแพ็คเกจท่องเที่ยว**: ร่วมมือกับร้านค้าและสถานที่ท่องเที่ยวในจังหวัดที่มีคนไปเที่ยวบ่อยเพื่อสร้างแพ็คเกจท่องเที่ยวที่ดึงดูดใจ
*   **โฆษณาในจุดหมายปลายทาง**: ใช้ข้อมูลเพื่อโฆษณาสถานที่ท่องเที่ยวและร้านค้าในจังหวัดที่นักท่องเที่ยวนิยมไปเพื่อเพิ่มการเข้าชมและรายได้ให้กับธุรกิจท้องถิ่น






In [70]:
# Example: Identifying popular destinations
# หากปทุมเป็นจังหวัดที่มีผู้มาเยือนบ่อยครั้ง เราสามารถโปรโมตแพ็คเกจท่องเที่ยวหรือสถานที่ท่องเที่ยวในท้องถิ่นในพื้นที่นั้นได้
popular_destinations = trips_df['province_list'].str.split(', ', expand=True).stack().value_counts().head(5)
print("Top 5 popular destinations:")
print(popular_destinations)
print()
print("สร้างแพ็คเกจท่องเที่ยวได้:")
print("ร่วมมือกับร้านค้าใน Pathum Thani เพื่อสร้างแพ็คเกจท่องเที่ยว")
print("ร่วมมือกับร้านค้าใน Nakhon Pathom เพื่อสร้างแพ็คเกจท่องเที่ยว")

Top 5 popular destinations:
Pathum Thani     2034
Nakhon Pathom    1791
Nonthaburi       1705
Bangkok          1648
Samut Sakhon     1532
Name: count, dtype: int64

สร้างแพ็คเกจท่องเที่ยวได้:
ร่วมมือกับร้านค้าใน Pathum Thani เพื่อสร้างแพ็คเกจท่องเที่ยว
ร่วมมือกับร้านค้าใน Nakhon Pathom เพื่อสร้างแพ็คเกจท่องเที่ยว


### 3. การให้ส่วนลดในช่วงนอกฤดูกาลท่องเที่ยว


*   **กระตุ้นการเดินทางในช่วงนอกฤดู**: ให้ส่วนลดหรือโปรโมชันพิเศษในช่วงที่คนไม่ค่อยเดินทาง เพื่อดึงดูดนักท่องเที่ยวให้มาเที่ยวมากขึ้น
*   **สร้างกิจกรรมพิเศษ**: จัดกิจกรรมหรือเทศกาลในช่วงนอกฤดูท่องเที่ยวเพื่อสร้างความน่าสนใจและดึงดูดนักท่องเที่ยว






In [72]:
# Example: Seasonal trends
# ให้ส่วนลดในช่วงนอกฤดูกาลท่องเที่ยวสามารถดึงดูดนักท่องเที่ยวมากขึ้นได้
trips_df['start_date'] = pd.to_datetime(trips_df['start_date'])
trips_df['month'] = trips_df['start_date'].dt.month
seasonal_trends = trips_df.groupby('month').size()
print("Seasonal travel trends:")
print(seasonal_trends)
print()
print("ให้ส่วนลดในช่วงนอกฤดูท่องเที่ยว:")
print("ให้ส่วนลด 10% สำหรับการเดินทางไป Chiang Mai ในช่วงนอกฤดูท่องเที่ยว")
print("ให้ส่วนลด 15% สำหรับการเดินทางไป Phra Nakhon Si Ayutthaya ในช่วงนอกฤดูท่องเที่ยว")
print("ให้ส่วนลด 20% สำหรับการเดินทางไป Bangkok ในช่วงนอกฤดูท่องเที่ยว")

Seasonal travel trends:
month
1     4231
2      134
3      124
4      161
5      160
6      228
7      247
8      280
9      328
10     480
11     746
12    3711
dtype: int64

ให้ส่วนลดในช่วงนอกฤดูท่องเที่ยว:
ให้ส่วนลด 10% สำหรับการเดินทางไป Chiang Mai ในช่วงนอกฤดูท่องเที่ยว
ให้ส่วนลด 15% สำหรับการเดินทางไป Phra Nakhon Si Ayutthaya ในช่วงนอกฤดูท่องเที่ยว
ให้ส่วนลด 20% สำหรับการเดินทางไป Bangkok ในช่วงนอกฤดูท่องเที่ยว
